# Model testing

- It's just a testing ground for the model. One model is loaded and a `cmc.test_model` is applied on it.
- As a result confusion matrix, ROC, AUROC, F1, PR curves will be generated and saved to the model's folder

In [1]:
import os
from pathlib import Path
import confinement_mode_classifier as cmc
import alt_models as am
import torchvision
import torch
from tqdm import tqdm
import torch.nn as nn
import re
import pandas as pd
path = Path(os.getcwd())
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
ris_option = 'both'
second_img_opt = None
num_workers = 32
batch_size = 32
test_df_contains_val_df = True
exponential_elm_decay = False 
num_classes = 3
test_run = False
grayscale = False

In [23]:

#### Create dataloaders ########################################
shot_usage = pd.read_csv(f'{path}/data/shot_usageNEW.csv')
shot_for_ris = shot_usage[shot_usage['used_for_ris2'] if ris_option == 'RIS2' else shot_usage['used_for_ris1']]
shot_numbers = shot_for_ris['shot']
shots_for_testing = shot_for_ris[shot_for_ris['used_as'] == 'test']['shot']
shots_for_validation = shot_for_ris[shot_for_ris['used_as'] == 'val']['shot']
shots_for_training = shot_for_ris[shot_for_ris['used_as'] == 'train']['shot']

if test_run:
   shots_for_testing = shots_for_testing[:3]
   shots_for_validation = shots_for_validation[:3]
   shots_for_training = shots_for_training[:3]

shot_df, test_df, val_df, train_df = cmc.load_and_split_dataframes(path,shot_numbers, shots_for_training, shots_for_testing, 
                                                               shots_for_validation, use_ELMS=num_classes==3, ris_option=ris_option,
                                                               exponential_elm_decay=exponential_elm_decay)

if ris_option == 'both':
   shot_for_ris2 = shot_usage[shot_usage['used_for_ris2']]
   shot_numbers_ris2 = shot_for_ris2['shot']
   shots_for_testing_ris2 = shot_for_ris2[shot_for_ris2['used_as'] == 'test']['shot']
   shots_for_validation_ris2 = shot_for_ris2[shot_for_ris2['used_as'] == 'val']['shot']
   shots_for_training_ris2 = shot_for_ris2[shot_for_ris2['used_as'] == 'train']['shot']

   if test_df_contains_val_df:
      shots_for_testing_ris2 = pd.concat([shots_for_testing_ris2, shots_for_validation_ris2])

   if test_run:
      shots_for_testing_ris2 = shots_for_testing_ris2[:3]
      shots_for_validation_ris2 = shots_for_validation_ris2[:3]
      shots_for_training_ris2 = shots_for_training_ris2[:3]

   shot_df_ris2, test_df_ris2, val_df_ris2, train_df_ris2 = cmc.load_and_split_dataframes(path,shot_numbers_ris2, shots_for_training_ris2, shots_for_testing_ris2, 
                                                                  shots_for_validation_ris2, use_ELMS=num_classes==3, ris_option='RIS2',
                                                                  exponential_elm_decay=exponential_elm_decay)

   test_df = pd.concat([test_df, test_df_ris2]).reset_index(drop=True)
   val_df = pd.concat([val_df, val_df_ris2]).reset_index(drop=True)
   train_df = pd.concat([train_df, train_df_ris2]).reset_index(drop=True)

shots_for_testing = pd.concat([shots_for_testing, shots_for_testing_ris2]).reset_index(drop=True)
shots_for_validation = pd.concat([shots_for_validation, shots_for_validation_ris2]).reset_index(drop=True)
shots_for_training = pd.concat([shots_for_training, shots_for_training_ris2]).reset_index(drop=True)

test_dataloader = cmc.get_dloader(test_df, path, batch_size, balance_data=False, 
                                      shuffle=False, num_workers=num_workers, 
                                      augmentation=False, grayscale=grayscale)

### Load the model

In [4]:
#Initiate a single camera model
pretrained_model = torchvision.models.resnet34(weights='IMAGENET1K_V1', )
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, 3) #3 classes: L-mode, H-mode, ELM
pretrained_model = pretrained_model.to(device)


if grayscale:
    # Luminance weights for RGB to grayscale conversion
    weights_rgb_to_gray = torch.tensor([0.2989, 0.5870, 0.1140]).view(1, 3, 1, 1).to(device)

    # Get the original weights of the first conv layer
    original_weights = pretrained_model.conv1.weight.data

    # Compute the weighted sum of the RGB channels
    grayscale_weights = (original_weights * weights_rgb_to_gray).sum(dim=1, keepdim=True)

    # Update the first convolutional layer
    pretrained_model.conv1 = nn.Conv2d(
        in_channels=1,
        out_channels=pretrained_model.conv1.out_channels,
        kernel_size=pretrained_model.conv1.kernel_size,
        stride=pretrained_model.conv1.stride,
        padding=pretrained_model.conv1.padding,
        bias=pretrained_model.conv1.bias is not None)

    # Assign the new grayscale weights to the first conv layer
    pretrained_model.conv1.weight = nn.Parameter(grayscale_weights)

    # If there is a bias term, keep it unchanged
    if pretrained_model.conv1.bias is not None:
        pretrained_model.conv1.bias = nn.Parameter(pretrained_model.conv1.bias.data)
        
#Which model to test?
model_path = f'{path}/runs/24-05-10, 19-50-51 both, NEW DATA, 3 classes, resnet34_all_layers'

#TEST one camera model
pretrained_model.load_state_dict(torch.load(f'{model_path}/model.pt'))
pretrained_model.eval()
pretrained_model.to(device)

#TEST ensembled model
# ensembled_model = cmc.TwoImagesModel(modelA=pretrained_model, modelB=pretrained_model, hidden_units=30).to(device)
# ensembled_model.load_state_dict(torch.load(f'{model_path}/model.pt'))
# ensembled_model.eval()
# ensembled_model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Test model on unbalanced test dset

In [15]:
test_dataloader.dataset.__len__()

75026

In [22]:
#metrics = cmc.test_model(f'{path}/RIS1 preds for doubtful shots', pretrained_model, test_dataloader_augmented, max_batch=0, comment='augmented_last_fc')
metrics = cmc.test_model(model_path, pretrained_model, test_dataloader, 
                         max_batch=0, comment='original_last_fc', signal_name='img', num_classes=3)

Processing batches:   0%|          | 10/2345 [00:07<28:52,  1.35it/s] 


KeyboardInterrupt: 

In [11]:
metrics['prediction_df'].to_csv(f'{model_path}/prediction_df.csv')

In [17]:
len(shots_for_testing)

15

In [24]:
img_path = cmc.per_shot_test(path=model_path, 
                                shots=shots_for_testing, results_df=metrics['prediction_df'], 
                                writer=None, num_classes=3, two_images=ris_option=='both')

  5%|▌         | 2/37 [00:00<00:06,  5.68it/s]/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
 38%|███▊      | 14/37 [00:02<00:03,  6.34it/s]/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/compass/Shared/Users/bogdanov/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:697: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
 46%|████▌     

### Here the model is tested on individual shots (generates time-confidence graph)

In [12]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(model_path)

In [ ]:
one_digit_metrics = {'Accuracy on test_dataset': metrics['accuracy'], 
                        'F1 metric on test_dataset':metrics['f1'].tolist(), 
                        'Precision on test_dataset':metrics['precision'].tolist(), 
                        'Recall on test_dataset':metrics['recall'].tolist()}

writer.add_hparams(hyperparameters, one_digit_metrics)
writer.close()

# Save hyperparameters and metrics to a JSON file
for key in ['shots_for_testing', 'shots_for_validation', 'shots_for_training']:
    hyperparameters[key] = hyperparameters[key].tolist()  # Convert tensors to lists
all_hparams = {**hyperparameters, **one_digit_metrics}
# Convert to JSON
json_str = json.dumps(all_hparams, indent=4)
with open(f'{path}/runs/{timestamp}_last_fc/hparams.json', 'w') as f:
    f.write(json_str)

In [7]:
one_digit_metrics = {'Accuracy on test_dataset': metrics['accuracy'], 
                        'F1 metric on test_dataset': metrics['f1'].tolist(), 
                        'Precision on test_dataset': metrics['precision'].tolist(), 
                        'Recall on test_dataset': metrics['recall'].tolist()}


In [8]:
import json
one_digit_metrics
json_str = json.dumps(one_digit_metrics, indent=4)

In [9]:
json_str

'{\n    "Accuracy on test_dataset": 0.9465918481593047,\n    "F1 metric on test_dataset": 0.9465918481593047,\n    "Precision on test_dataset": 0.9465918481593047,\n    "Recall on test_dataset": 0.9465918481593047\n}'

In [10]:
shots_for_testing = list(map(int,shots_for_testing))
#img_path = cmc.per_shot_test(path=f'{path}/RIS1 preds for doubtful shots', shots=shots_for_testing, results_df=metrics['prediction_df'])